In [ ]:
#| default_exp conv

In [ ]:
#| export
from torch import nn

In [ ]:
#| hide
from nbdev.showdoc import *

## Core

In [ ]:
#| export
class ConvNormAct(nn.Sequential):
    def __init__(
        self,
        in_channels,
        out_channels,
        kernel_size=3,
        bias=True,
        stride=2,
        act=True
    ):
        super().__init__(
            nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=kernel_size//2, bias=bias),
            nn.BatchNorm2d(out_channels),
            nn.ReLU() if act else None
        )

## Resnet

In [ ]:
#| export
class ResnetStem():
    def __init__(
        self,
        stem_sizes
    ):
        self.conv1 = nn.Sequential(
            [
                ConvNormAct(
                    in_channels=stem_sizes[i],
                    out_channels=stem_sizes[i+1],
                    kernel_size=3,
                    stride=2 if i==0 else 1
                )
            for i in stem_sizes
            ]
        )
            
        super().init(
            ConvNormAct(in_channels, out_channels, kernel_size=kernel_size, stride=stride),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

In [ ]:
#| export
class BottleneckBlock(nn.Module):
    def __init__(
        self,
        in_channels,
        out_channels,
        reduction=4,
        stride=1
    ):
        
        reduced_features = out_channels // reduction
        
        self.block = nn.Sequential(
            ConvNormAct(in_channels, reduced_features, kernel_size=1, stride=stride), # <----- including stride enables us to stride on this layer
            ConvNormAct(reduced_features, reduced_features, kernel_size=3),
            ConvNormAct(reduced_features, out_channels, kernel_size=1)
        )
        
    def forward(self, x):
        x = self.block(x)
        return x

In [ ]:
#| export
class BottleneckBlock(nn.Module):
    def __init__(
        self,
        in_channels,
        out_channels,
        reduction=4,
        stride=1
    ):
        
        reduced_features = out_channels // reduction
        
        self.block = nn.Sequential(
            ConvNormAct(in_channels, reduced_features, kernel_size=1, stride=stride), # <----- including stride enables us to stride on this layer
            ConvNormAct(reduced_features, reduced_features, kernel_size=3),
            ConvNormAct(reduced_features, out_channels, kernel_size=1)
        )
        
        self.shortcut = (
            nn.Sequential(
                ConvNormAct(in_channels, out_channels, kernel_size=1)
            ) if in_channels != out_channels else nn.Identity()
        )
        
        self.pool = (
            nn.AvgPool2d(kernel_size=3, stride=stride, padding=1) 
            if stride != 1 else nn.Identity()
        )
        
    def forward(self, x):
        residual = x
        x = self.block(x)
        residual = self.shortcut(self.pool(residual))
        x += residual
        return x

In [ ]:
#| export
class ResnetStage(nn.Sequential):
    def __init__(
        self,
        in_channels,
        out_channels,
        depth,
        stride=2
    ):
        super().__init__(
            BottleneckBlock(in_channels, out_channels, stride=2),
            *[
                BottleneckBlock(out_channels, out_channels)
                for i in range(depth - 1)
            ]
        )

In [ ]:
#| export
class ResnetNN(nn.Module):
    def __init__(
        self,
        img_channels,
        stem_sizes,
        widths,
        depths,
        num_classes
    ):
        stem_sizes = [img_channels, *stem_sizes]
        self.stem = ResnetStem(stem_sizes)
        
        self.stages = nn.ModuleList(
            [
                ResnetStage(stem_sizes[-1], widths[0], depths[0], stride=1),
                *[
                    ResnetStage(widths[i], widths[i+1], depths[i+1])
                    for i in widths
                ]
            ]   
        )
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(widths[-1], num_classes)
        
    def forward(self, x):
        x = self.stem(x)
        for stage in self.stages:
            x = stage(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

## Export

In [ ]:
import miniai

In [ ]:
miniai

<module 'miniai' (namespace)>

In [ ]:
from miniai.learner import *

ModuleNotFoundError: No module named 'miniai.learner'